In [78]:
# -*- coding: utf-8 -*-
"""
Created on Mon Jul 11 18:06:00 2022

@author: Beau.Uriona
"""

import os
import folium
import numpy as np
import pandas as pd
import geopandas as gpd
from requests import Session

# Import the function to get connect to the db
from snowexsql.db import get_db
from snowexsql.conversions import query_to_pandas
from snowexsql.data import SiteData, PointData, LayerData, ImageData

BUFFER_IN_METERS = 10 * 1000 # 10km
SNOW_EX_GEOJSON_FILENAME = "snow_ex_all_stations.geojson"
REFRESH_SNOW_EX_GEOJSON = False
SNOWEX_DB_NAME = 'snow:hackweek@db.snowexdata.org/snowex'

if REFRESH_SNOW_EX_GEOJSON or not os.path.isfile(SNOW_EX_GEOJSON_FILENAME):
    # Get the snow ex sites
    engine, session = get_db(SNOWEX_DB_NAME)
    qry = session.query(SiteData.site_name, SiteData.site_id, SiteData.latitude, SiteData.longitude)

    df_snow_ex = query_to_pandas(qry, engine).drop_duplicates(subset=["site_id"])
    gdf_snow_ex = gpd.GeoDataFrame(
        df_snow_ex, 
        geometry=gpd.points_from_xy(df_snow_ex.longitude, df_snow_ex.latitude),
        crs=4326
    )
    gdf_snow_ex.to_file(SNOW_EX_GEOJSON_FILENAME, driver='GeoJSON')
    #gdf_snow_ex.rename(columns={"geom": "geometry"}, inplace=True)
    session.close()
else:
    gdf_snow_ex = gpd.read_file(SNOW_EX_GEOJSON_FILENAME)
    
print(f"The CRS of the Snow Ex metadata is - {gdf_snow_ex.crs}")
gdf_snow_ex_buffer = gdf_snow_ex.to_crs(26912).buffer(BUFFER_IN_METERS).to_crs(4326)
snow_ex_bounds = gdf_snow_ex.to_crs(4326).total_bounds
print(f"The bounding box of the snow Ex sites is - {snow_ex_bounds}")
snow_ex_map = gdf_snow_ex_buffer.to_crs(4326).explore()
snow_ex_map = gdf_snow_ex.to_crs(4326).explore(m=snow_ex_map)


# Get SNOTEL Sites
AWDB_API_DOMAIN = "https://api.snowdata.info"
sntl_meta_url = f"{AWDB_API_DOMAIN}/stations/getMeta?network=SNTL&format=geojson"
msnt_meta_url = f"{AWDB_API_DOMAIN}/stations/getMeta?network=MSNT&format=geojson"
# Get the data and rename everything to match the snow ex columns
gdf_sntl = gpd.read_file(sntl_meta_url).rename(
    columns={"name": "site_name", "stationTriplet": "site_id"}
)
gdf_msnt = gpd.read_file(msnt_meta_url).rename(
    columns={"name": "site_name", "stationTriplet": "site_id"}
)
gdf_sntl = pd.concat([gdf_msnt, gdf_sntl])
gdf_sntl = gdf_sntl[gdf_sntl["beginDate"].dt.year <= 2017]
print(f"The CRS of the Snotel metadata is - {gdf_sntl.crs}, better change it")
gdf_sntl = gdf_sntl.to_crs(gdf_snow_ex.crs)
print(f"The CRS of the Snotel metadata is now - {gdf_sntl.crs}, all good!")
# Only keep the columns we care about... for now
gdf_sntl = gdf_sntl.drop(
    columns=[i for i in gdf_sntl.columns if i not in ["geometry", "site_id", "site_name"]]
).set_geometry("geometry")
gdf_sntl_clipped = gdf_sntl.clip(gdf_snow_ex_buffer)
sntl_map = gdf_sntl_clipped.explore()
snow_ex_map = gdf_sntl_clipped.explore(m=snow_ex_map)
print(f"{len(gdf_sntl_clipped)} AWDB sites were found within {BUFFER_IN_METERS / 1000:.0f} km of SnowEx Sites")
_ = [print(f'* {row["site_name"]} ({row["site_id"]})') for idx, row in gdf_sntl_clipped.sort_values("site_name").iterrows()]
valid_triplets = list(set(gdf_sntl_clipped["site_id"]))

The CRS of the Snow Ex metadata is - epsg:4326
The bounding box of the snow Ex sites is - [-120.29898   35.85793 -105.54616   44.30447]
The CRS of the Snotel metadata is - epsg:4979, better change it
The CRS of the Snotel metadata is now - epsg:4326, all good!
28 AWDB sites were found within 10 km of SnowEx Sites
* Banner Summit (312:ID:SNTL)
* Bogus Basin (978:ID:SNTL)
* Brighton (366:UT:SNTL)
* Butte (380:CO:SNTL)
* Caples Lake (CAP:CA:MSNT)
* Carson Pass (1067:CA:SNTL)
* Devils Postpile (DPO:CA:MSNT)
* Fool Creek (1186:CO:SNTL)
* Forestdale Creek (1049:CA:SNTL)
* Idarado (538:CO:SNTL)
* Independence Camp (539:CA:SNTL)
* Independence Creek (540:CA:SNTL)
* Independence Lake (541:CA:SNTL)
* Joe Wright (551:CO:SNTL)
* Mammoth Pass (MHP:CA:MSNT)
* Mesa Lakes (622:CO:SNTL)
* Mill-D North (628:UT:SNTL)
* Mineral Creek (629:CO:SNTL)
* Mores Creek Summit (637:ID:SNTL)
* Niwot (663:CO:SNTL)
* Red Mountain Pass (713:CO:SNTL)
* Sawtooth (1251:CO:SNTL)
* Schneiders (SCN:CA:MSNT)
* Schofield Pass

In [39]:
from datetime import datetime

import requests
import pandas as pd
import numpy as np

API_DOMAIN = "https://api.snowdata.info/"

def get_awdb_data(site_ids, element="WTEQ", sdate=datetime(1899,10,1), edate=datetime.now(), orient="records", server=API_DOMAIN, sesh=None):
    """
    Takes a list of site ids or a single site id and by default returns SWE period of record data as a single or list of dataframes,
    but user can pass args to modify data returned.
    
    Valid elements include WTEQ, SNWD, PREC, SMS, STO, TAVG
    site_id takes the form of a triplet made from <network_site_id>:<state_abbrv>:<network> where network is either SNTL or MNST
    """
    dfs = []
    return_single = False
    if not isinstance(site_ids, list):
        site_ids = [site_ids]
        return_single = True
    for site_id in site_ids:
        endpoint = "data/getDaily"
        date_args = f"sDate={sdate:%Y-%m-%d}&eDate={edate:%Y-%m-%d}"
        frmt_args = f"format=json&orient={orient}"
        all_args = f"?triplet={site_id}&{date_args}&element={element}&{frmt_args}"
        url = f"{server}{endpoint}{all_args}"
        print(f"getting data for {site_id} {element} starting {sdate:%Y-%m-%d} and ending {edate:%Y-%m-%d}")
        data_col_lbl = f"{site_id}" + ":" + f"{element}"
        if sesh:
            req = sesh.get(url)
        else:
            req = requests.get(url)
        if req.ok:
            df = pd.DataFrame.from_dict(req.json())
            df.columns = ["Date", data_col_lbl]
            df.set_index("Date", inplace=True)
        else:
            print("  No data returned!")
            df = pd.DataFrame(
                data=[
                    {"Date": pd.NaT, data_col_lbl: np.nan}
                ],
            ).set_index("Date").dropna()
        dfs.append(df)
    if return_single:
        return dfs[0]
    return dfs

with requests.Session() as sesh:
    dfs_swe = get_awdb_data(site_ids=valid_triplets, element="WTEQ", edate=datetime(2022, 4, 7),sdate=datetime(2022, 4, 1), sesh=sesh)
    dfs_snwd = get_awdb_data(site_ids=valid_triplets, element="SNWD", edate=datetime(2022, 4, 7),sdate=datetime(2022, 4, 1), sesh=sesh)
all_swe_data = pd.concat(dfs_swe, axis=1)
all_snwd_data = pd.concat(dfs_snwd, axis=1)
all_awdb_data = pd.concat([all_swe_data, all_snwd_data], axis=1).fillna(np.nan)
all_awdb_data.tail(2)


getting data for 1067:CA:SNTL WTEQ starting 2022-04-01 and ending 2022-04-07
getting data for 629:CO:SNTL WTEQ starting 2022-04-01 and ending 2022-04-07
getting data for 838:CO:SNTL WTEQ starting 2022-04-01 and ending 2022-04-07
getting data for 622:CO:SNTL WTEQ starting 2022-04-01 and ending 2022-04-07
getting data for 814:UT:SNTL WTEQ starting 2022-04-01 and ending 2022-04-07
getting data for SCN:CA:MSNT WTEQ starting 2022-04-01 and ending 2022-04-07
getting data for 312:ID:SNTL WTEQ starting 2022-04-01 and ending 2022-04-07
getting data for 663:CO:SNTL WTEQ starting 2022-04-01 and ending 2022-04-07
getting data for 380:CO:SNTL WTEQ starting 2022-04-01 and ending 2022-04-07
getting data for 1186:CO:SNTL WTEQ starting 2022-04-01 and ending 2022-04-07
getting data for 1251:CO:SNTL WTEQ starting 2022-04-01 and ending 2022-04-07
getting data for 766:UT:SNTL WTEQ starting 2022-04-01 and ending 2022-04-07
getting data for 628:UT:SNTL WTEQ starting 2022-04-01 and ending 2022-04-07
getting d

,1067:CA:SNTL:WTEQ,629:CO:SNTL:WTEQ,838:CO:SNTL:WTEQ,622:CO:SNTL:WTEQ,814:UT:SNTL:WTEQ,SCN:CA:MSNT:WTEQ,312:ID:SNTL:WTEQ,663:CO:SNTL:WTEQ,380:CO:SNTL:WTEQ,1186:CO:SNTL:WTEQ,...,540:CA:SNTL:SNWD,737:CO:SNTL:SNWD,539:CA:SNTL:SNWD,MHP:CA:MSNT:SNWD,538:CO:SNTL:SNWD,637:ID:SNTL:SNWD,713:CO:SNTL:SNWD,541:CA:SNTL:SNWD,366:UT:SNTL:SNWD,551:CO:SNTL:SNWD
Date,,,,,,,,,,,,,,,,,,,,,
2022-04-06,10.4,11.2,15.2,15.8,17.5,NaN,19.0,12.4,12.7,17.5,...,3.0,83.0,13.0,NaN,34.0,40.0,62.0,72.0,33.0,60.0
2022-04-07,9.6,11.2,15.3,16.0,17.5,NaN,19.0,12.5,13.2,17.5,...,1.0,83.0,10.0,NaN,34.0,39.0,62.0,71.0,32.0,59.0


In [ ]:
engine, session = get_db(SNOWEX_DB_NAME)
use_cols = ("site_id", "date", "pit_id", "value", "geometry", "depth", "instrument")
for awdb_site in valid_triplets:
    gdf_awdb_site_buffer = gdf_sntl[gdf_sntl["site_id"] == awdb_site].to_crs(26912).buffer(BUFFER_IN_METERS).to_crs(4326)
    gdf_snow_ex_inside = gdf_snow_ex.clip(gdf_awdb_site_buffer)
    snow_ex_site_list = gdf_snow_ex_inside["site_id"].tolist()
    awdb_site_name = gdf_sntl[gdf_sntl["site_id"] == awdb_site]["site_name"].values[0]
    print(
        f"The AWDB site: {awdb_site_name} has {len(snow_ex_site_list)} SnowEx pits within "
        f"{BUFFER_IN_METERS / 1000:.0f} km ({', '.join(snow_ex_site_list)})."
    )
    
    qry = session.query(LayerData).filter(LayerData.site_id.in_(snow_ex_site_list)).filter(LayerData.type == "density").limit(200)
    df_snow_ex_data = query_to_pandas(qry, engine)
    gdf_snow_ex_data = gpd.GeoDataFrame(
        df_snow_ex_data, 
        geometry=gpd.points_from_xy(df_snow_ex_data.longitude, df_snow_ex_data.latitude),
        crs=4326
    )
    gdf_snow_ex_data.drop(columns=[i for i in df_snow_ex_data.columns if i not in use_cols], inplace=True)
    gdf_snow_ex_data.loc[:, "value"] = gdf_snow_ex_data["value"].astype(float)
    gdf_snow_ex_data.index = pd.to_datetime(gdf_snow_ex_data["date"])
    df_grp_by = gdf_snow_ex_data.groupby(by=gdf_snow_ex_data.index)
    df_bulk_density = df_grp_by["value"].mean() / 1000 # convert kg/m^3
    df_depth = df_grp_by["depth"].max()
    df_swe = df_depth * df_bulk_density * 10 # convert cm to mm
    df_values = pd.DataFrame(data={"swe": swe.round(0), "depth": depth})
    gdf_snow_ex_data.drop(columns="date", inplace=True)
    df_pit_data = df_grp_by.first().drop(columns=["depth", "value"])
    df_combined = df_pit_data.join(df_values)
    break
df_combined